# **Cálculo de Poblaciones Trimestrales para Análisis de Natalidad**

Este notebook genera el dataset de poblaciones trimestrales departamentales necesario para calcular las tasas de natalidad en el análisis principal. A partir de datos poblacionales anuales, implementa un algoritmo de interpolación lineal que estima la población para cada trimestre, considerando las tasas de crecimiento histórico por departamento. Incluye cálculo de tasas de crecimiento trimestral y verificación de calidad de datos, proporcionando los denominadores poblacionales requeridos para el cálculo preciso de las tasas de natalidad trimestrales por departamento.

## **1. Configuración y Carga de Datos Poblacionales**

In [ ]:
# Librerías
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
# Cargar dataset principal de poblaciones departamentales
df = pd.read_csv('poblaciones-departamentales.csv')

In [ ]:
# Limpiar nombres de columnas (eliminar espacios)
df.columns = df.columns.str.strip()

In [ ]:
print(f"Registros totales: {len(df)}")
print(f"Departamentos únicos: {df['Departamento'].nunique()}")
print(f"Rango de años: {df['Año'].min()} - {df['Año'].max()}")
print(f"Columnas disponibles: {list(df.columns)}")

Registros totales: 425
Departamentos únicos: 25
Rango de años: 2009 - 2025
Columnas disponibles: ['Año', 'Departamento', 'Poblacion total']


In [ ]:
print("Primeras filas del dataset:")
print(df.head())

Primeras filas del dataset:
    Año Departamento  Poblacion total
0  2009     Amazonas           407451
1  2009       Áncash          1096786
2  2009     Apurímac           431180
3  2009     Arequipa          1208011
4  2009     Ayacucho           646351


## **2. Generación de Poblaciones Trimestrales por Departamento**

In [ ]:
# Función para generar datos trimestrales mediante interpolación lineal
def generar_datos_trimestrales(df):
    """
    Genera poblaciones trimestrales para cada departamento usando interpolación lineal
    Asume crecimiento constante entre observaciones anuales.
    """
    resultados = []

    # Procesar cada departamento
    for dept in df['Departamento'].unique():
        df_dept = df[df['Departamento'] == dept].sort_values('Año').reset_index(drop=True)

        for i in range(len(df_dept)):
            año = df_dept.loc[i, 'Año']
            pob_inicio = df_dept.loc[i, 'Poblacion total']

            # Calcular tasa de crecimiento anual
            if i < len(df_dept) - 1:
                # Usar crecimiento real entre años consecutivos
                pob_fin = df_dept.loc[i + 1, 'Poblacion total']
                tasa_anual = (pob_fin - pob_inicio) / pob_inicio
            else:
                # Para el último año (2025), estimar usando promedio de últimos 3 años
                if i >= 3:
                    tasas_recientes = []
                    for j in range(max(0, i-3), i):
                        pob_j = df_dept.loc[j, 'Poblacion total']
                        pob_j_sig = df_dept.loc[j + 1, 'Poblacion total']
                        tasa_j = (pob_j_sig - pob_j) / pob_j
                        tasas_recientes.append(tasa_j)
                    tasa_anual = np.mean(tasas_recientes)
                else:
                    # Si no hay datos históricos suficientes
                    tasa_anual = 0

            # Generar población para cada trimestre del año
            for trimestre in range(1, 5):
                fraccion_año = (trimestre-1) / 4 # Q1=0, Q2=0.25, Q3=0.5, Q4=0.75
                pob_trimestre = pob_inicio * (1 + tasa_anual * fraccion_año)

                resultados.append({
                    'Año': año,
                    'Trimestre': f'T{trimestre}',
                    'Departamento': dept,
                    'Poblacion_Trimestral': int(round(pob_trimestre))
                })

    return pd.DataFrame(resultados)

In [ ]:
# Generar datos trimestrales
df_trimestral = generar_datos_trimestrales(df)

In [ ]:
print(f"Poblaciones trimestrales generadas: {len(df_trimestral)} registros")
print(f"Período cubierto: {df_trimestral['Año'].min()}-{df_trimestral['Año'].max()}")

Poblaciones trimestrales generadas: 1700 registros
Período cubierto: 2009-2025


## **3. Cálculo de Tasas de Crecimiento Trimestral**

In [ ]:
# Función para calcular tasas de crecimiento entre trimestres consecutivos
def calcular_tasa_trimestral(grupo):
    """
    Calcula la tasa de crecimiento porcentual entre trimestres consecutivos para cada departamento
    """
    grupo = grupo.sort_values(['Año', 'Trimestre']).reset_index(drop=True)
    grupo['Tasa_Crecimiento_Trimestral_%'] = (
        (grupo['Poblacion_Trimestral'].diff() / grupo['Poblacion_Trimestral'].shift(1)) * 100
    )
    return grupo

In [ ]:
# Aplicar cálculo por departamento
df_trimestral = df_trimestral.groupby('Departamento', group_keys=False).apply(calcular_tasa_trimestral)

/tmp/ipython-input-2490345764.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_trimestral = df_trimestral.groupby('Departamento', group_keys=False).apply(calcular_tasa_trimestral)


In [ ]:
# Crear columna de periodo para ordenamiento y referencia
df_trimestral['Periodo'] = df_trimestral['Año'].astype(str) + '-' + df_trimestral['Trimestre']

In [ ]:
# Filtrar solo datos desde 2015 hasta 2025 para coincidir con datos de natalidad
df_trimestral = df_trimestral[(df_trimestral['Año'] >= 2015) & (df_trimestral['Año'] <= 2025)]

## **4. Análisis Exploratorio**

In [ ]:
# Mostrar ejemplos de departamentos representativos
print("="*70)
print("POBLACIONES Y TASAS DE CRECIMIENTO TRIMESTRALES")
print("="*70)

departamentos_ejemplo = ['Lima', 'Arequipa', 'Cusco', 'Huancavelica', 'Madre de Dios']

print("\nÚLTIMOS 8 TRIMESTRES POR DEPARTAMENTO:")

for dept in departamentos_ejemplo:
    print(f"\n{dept}:")
    dept_data = df_trimestral[df_trimestral['Departamento'] == dept].tail(8)[
        ['Año', 'Trimestre', 'Poblacion_Trimestral', 'Tasa_Crecimiento_Trimestral_%']
    ]
    print(dept_data.to_string(index=False))
    print("-" * 70)

POBLACIONES Y TASAS DE CRECIMIENTO TRIMESTRALES

ÚLTIMOS 8 TRIMESTRES POR DEPARTAMENTO:

Lima:
 Año Trimestre  Poblacion_Trimestral  Tasa_Crecimiento_Trimestral_%
2024        T1              11304993                       0.348553
2024        T2              11344244                       0.347201
2024        T3              11383494                       0.345990
2024        T4              11422744                       0.344797
2025        T1              11461995                       0.343621
2025        T2              11502796                       0.355968
2025        T3              11543597                       0.354705
2025        T4              11584397                       0.353443
----------------------------------------------------------------------

Arequipa:
 Año Trimestre  Poblacion_Trimestral  Tasa_Crecimiento_Trimestral_%
2024        T1               1605569                       0.398513
2024        T2               1611961                       0.398114
2024   

In [ ]:
# Estadísticas por departamento
print("ESTADÍSTICAS DE TASAS DE CRECIMIENTO TRIMESTRAL POR DEPARTAMENTO")
print("="*70)

estadisticas = df_trimestral.groupby('Departamento')['Tasa_Crecimiento_Trimestral_%'].agg([
    ('Promedio_%', 'mean'),
    ('Mínimo_%', 'min'),
    ('Máximo_%', 'max'),
    ('Desv_Std_%', 'std'),
    ('N_Trimestres', 'count')
]).round(4)

print(estadisticas.sort_values('Promedio_%', ascending=False))

ESTADÍSTICAS DE TASAS DE CRECIMIENTO TRIMESTRAL POR DEPARTAMENTO
                         Promedio_%  Mínimo_%  Máximo_%  Desv_Std_%  \
Departamento                                                          
Madre de Dios                0.8611    0.7407    1.0153      0.0961   
Ica                          0.6086    0.4920    0.7583      0.0963   
Ucayali                      0.5840    0.4679    0.7332      0.0963   
Arequipa                     0.5086    0.3934    0.6566      0.0964   
Prov. Const. del Callao      0.4991    0.3840    0.6470      0.0965   
Tacna                        0.4835    0.3687    0.6311      0.0964   
Lima                         0.4582    0.3436    0.6055      0.0965   
Tumbes                       0.4374    0.3231    0.5839      0.0962   
La Libertad                  0.4141    0.3000    0.5606      0.0964   
San Martín                   0.3828    0.2693    0.5287      0.0963   
Piura                        0.3760    0.2622    0.5219      0.0964   
Moquegua    

In [ ]:
# Verificación de calidad de datos
print(f"Registros totales: {len(df_trimestral):,}")
print(f"Departamentos procesados: {df_trimestral['Departamento'].nunique()}")
print(f"Registros con población nula: {df_trimestral['Poblacion_Trimestral'].isnull().sum()}")
print(f"Registros con tasa de crecimiento nula: {df_trimestral['Tasa_Crecimiento_Trimestral_%'].isnull().sum()}")
print(f"Rango de población: {df_trimestral['Poblacion_Trimestral'].min():,} - {df_trimestral['Poblacion_Trimestral'].max():,}")

Registros totales: 1,100
Departamentos procesados: 25
Registros con población nula: 0
Registros con tasa de crecimiento nula: 0
Rango de población: 143,639 - 11,584,397


In [ ]:
df_trimestral.head()

,Año,Trimestre,Departamento,Poblacion_Trimestral,Tasa_Crecimiento_Trimestral_%,Periodo
24,2015,T1,Amazonas,408751,0.082759,2015-T1
25,2015,T2,Amazonas,409444,0.169541,2015-T2
26,2015,T3,Amazonas,410138,0.169498,2015-T3
27,2015,T4,Amazonas,410832,0.169211,2015-T4
28,2016,T1,Amazonas,411525,0.168682,2016-T1


## **5. Exportación del Dataset Final**

In [ ]:
# Exportar dataset final para uso en análisis de natalidad
df_trimestral.to_csv('poblaciones_trimestrales_con_tasas.csv', index=False)

print("Dataset 'poblaciones_trimestrales_con_tasas.csv' exportado correctamente.")

files.download('poblaciones_trimestrales_con_tasas.csv')

print("\n" + "=" * 85)
print("Este dataset será utilizado para calcular tasas de natalidad en el notebook principal")
print("=" * 85)

Dataset 'poblaciones_trimestrales_con_tasas.csv' exportado correctamente.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Este dataset será utilizado para calcular tasas de natalidad en el notebook principal
